In [65]:
import pandas as pd
import numpy as np
import json
import tqdm

In [66]:
game_ids_df = pd.read_excel("../data/game_ids.xlsx")
game_ids_df = game_ids_df.astype({"tournamentId":"str", "matchId":"str", "gameId":"str"})
game_ids_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9287 entries, 0 to 9286
Data columns (total 37 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   startTime(match)         9287 non-null   object 
 1   tournamentId             9287 non-null   object 
 2   blockName                9287 non-null   object 
 3   leagueName               9287 non-null   object 
 4   matchId                  9287 non-null   object 
 5   bestof                   9287 non-null   int64  
 6   blueteam_name            9287 non-null   object 
 7   blueteam_code            9287 non-null   object 
 8   blueteam_win             9287 non-null   int64  
 9   redteam_name             9287 non-null   object 
 10  redteam_code             9287 non-null   object 
 11  redteam_win              9287 non-null   int64  
 12  gameNumberInAMatch       9287 non-null   int64  
 13  gameId                   9287 non-null   object 
 14  patch                   

In [67]:
game_ids_with_picks_df = game_ids_df[pd.isna(game_ids_df["pick_0"]) == False].copy()
game_ids_with_picks_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8023 entries, 0 to 9286
Data columns (total 37 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   startTime(match)         8023 non-null   object 
 1   tournamentId             8023 non-null   object 
 2   blockName                8023 non-null   object 
 3   leagueName               8023 non-null   object 
 4   matchId                  8023 non-null   object 
 5   bestof                   8023 non-null   int64  
 6   blueteam_name            8023 non-null   object 
 7   blueteam_code            8023 non-null   object 
 8   blueteam_win             8023 non-null   int64  
 9   redteam_name             8023 non-null   object 
 10  redteam_code             8023 non-null   object 
 11  redteam_win              8023 non-null   int64  
 12  gameNumberInAMatch       8023 non-null   int64  
 13  gameId                   8023 non-null   object 
 14  patch                    8023

In [68]:
crawled_df = pd.read_excel("../data/crawling_result_with_codenames.xlsx")
crawled_df = crawled_df.astype({"tournament_id":"str", "patch":"str"})
crawled_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20179 entries, 0 to 20178
Data columns (total 41 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   date              20179 non-null  object
 1   tournament_title  20179 non-null  object
 2   patch             20179 non-null  object
 3   blueteam          20179 non-null  object
 4   redteam           20179 non-null  object
 5   winner_side       20179 non-null  object
 6   ban_0             20135 non-null  object
 7   ban_1             20152 non-null  object
 8   ban_2             20147 non-null  object
 9   ban_3             16973 non-null  object
 10  ban_4             16887 non-null  object
 11  ban_5             20127 non-null  object
 12  ban_6             20151 non-null  object
 13  ban_7             20143 non-null  object
 14  ban_8             16978 non-null  object
 15  ban_9             16875 non-null  object
 16  pick_0            20179 non-null  object
 17  pick_1      

In [69]:
pick_column_list = [f"pick_{x}" for x in range(10)]
left_on = ["tournamentId", "teamcode_blue_on_detail", "teamcode_red_on_detail"]
right_on = ["tournament_id", "blueteam_code", "redteam_code"]
left_on.extend(pick_column_list)
right_on.extend(pick_column_list)
game_ids_with_picks_df = game_ids_df.astype({"patch":"str"})
print(left_on)
print(right_on)




['tournamentId', 'teamcode_blue_on_detail', 'teamcode_red_on_detail', 'pick_0', 'pick_1', 'pick_2', 'pick_3', 'pick_4', 'pick_5', 'pick_6', 'pick_7', 'pick_8', 'pick_9']
['tournament_id', 'blueteam_code', 'redteam_code', 'pick_0', 'pick_1', 'pick_2', 'pick_3', 'pick_4', 'pick_5', 'pick_6', 'pick_7', 'pick_8', 'pick_9']


In [70]:
game_ids_with_picks_df[left_on].head()

,tournamentId,teamcode_blue_on_detail,teamcode_red_on_detail,pick_0,pick_1,pick_2,pick_3,pick_4,pick_5,pick_6,pick_7,pick_8,pick_9
0,112099354464733424,FLY,PSG,Urgot,Sejuani,Taliyah,Jinx,Nautilus,Rumble,JarvanIV,Azir,Senna,TahmKench
1,112099354464733424,FLY,PSG,Urgot,Maokai,Taliyah,Lucian,Nami,KSante,Sejuani,Azir,Kalista,Ashe
2,112099354464733424,PSG,FLY,KSante,XinZhao,Karma,Kalista,Rell,Olaf,Sejuani,Orianna,Varus,Nautilus
3,112099354464733424,T1,EST,KSante,LeeSin,Orianna,Kalista,Neeko,Yone,Volibear,Ahri,Draven,Rell
4,112099354464733424,EST,T1,Aatrox,LeeSin,Ahri,Draven,Ashe,KSante,Viego,Orianna,Kalista,Rumble


In [71]:
crawled_df[right_on].head()

,tournament_id,blueteam_code,redteam_code,pick_0,pick_1,pick_2,pick_3,pick_4,pick_5,pick_6,pick_7,pick_8,pick_9
0,112099354464733424,GEN,BLG,Rumble,Lee Sin,Azir,Senna,Maokai,Camille,Nidalee,Tristana,Zeri,Nautilus
1,112099354464733424,BLG,GEN,Jax,Nidalee,Jayce,Senna,Tahm Kench,Rumble,Lee Sin,Hwei,Kalista,Ashe
2,112099354464733424,BLG,GEN,Camille,Xin Zhao,Orianna,Varus,Ashe,Zac,Sejuani,Yone,Kalista,Blitzcrank
3,112099354464733424,GEN,BLG,K'Sante,Karthus,Yone,Senna,Tahm Kench,Twisted Fate,Xin Zhao,Neeko,Lucian,Nami
4,112099354464733424,BLG,T1,Camille,Xin Zhao,Neeko,Senna,Ornn,K'Sante,Viego,Taliyah,Varus,Nautilus


In [72]:
picks_of_game_ids_set = set(np.concatenate(game_ids_with_picks_df.loc[:,pick_column_list].values.tolist()).tolist())
len(picks_of_game_ids_set)

165

In [73]:
picks_of_crawled_set = set(np.concatenate(crawled_df.loc[:,pick_column_list].values.tolist()).tolist())
len(picks_of_crawled_set)

165

In [74]:
difference_of_game_ids = picks_of_game_ids_set.difference(picks_of_crawled_set)
difference_of_crawled = picks_of_crawled_set.difference(picks_of_game_ids_set)
print(difference_of_game_ids)
print(len(difference_of_game_ids))
print(difference_of_crawled)
print(len(difference_of_crawled))



{'Belveth', 'Leblanc', 'MissFortune', 'AurelionSol', 'XinZhao', 'JarvanIV', 'Renata', 'RekSai', 'Kaisa', 'FiddleSticks', 'MonkeyKing', 'Khazix', 'Chogath', 'TwistedFate', 'TahmKench', 'DrMundo', 'KSante', 'nan', 'Velkoz', 'LeeSin', 'KogMaw'}
21
{"Vel'Koz", "Kha'Zix", "Kog'Maw", 'Master Yi', 'Twisted Fate', "Kai'Sa", 'Miss Fortune', "Bel'Veth", 'Jarvan IV', 'Lee Sin', 'Aurelion Sol', 'Tahm Kench', "K'Sante", 'Xin Zhao', 'Fiddlesticks', 'Dr. Mundo', "Cho'Gath", "Rek'Sai", 'Renata Glasc', 'Wukong', 'LeBlanc'}
21


In [75]:
with open('../data/pick_name_not_the_same.json', 'r') as f:
    pick_difference_map = json.load(f)
crawled_df[pick_column_list] = crawled_df[pick_column_list].replace(pick_difference_map)

In [76]:
picks_of_game_ids_set = set(np.concatenate(game_ids_with_picks_df.loc[:,pick_column_list].values.tolist()).tolist())
picks_of_crawled_set = set(np.concatenate(crawled_df.loc[:,pick_column_list].values.tolist()).tolist())
difference_of_game_ids = picks_of_game_ids_set.difference(picks_of_crawled_set)
difference_of_crawled = picks_of_crawled_set.difference(picks_of_game_ids_set)
print(difference_of_game_ids)
print(len(difference_of_game_ids))
print(difference_of_crawled)
print(len(difference_of_crawled))


{'nan'}
1
{'Master Yi'}
1


In [77]:
result_df = pd.merge(game_ids_with_picks_df, crawled_df, how = "left", left_on = left_on, right_on = right_on)

In [78]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9383 entries, 0 to 9382
Data columns (total 68 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   startTime(match)         9383 non-null   object
 1   tournamentId             9383 non-null   object
 2   blockName                9383 non-null   object
 3   leagueName               9383 non-null   object
 4   matchId                  9383 non-null   object
 5   bestof                   9383 non-null   int64 
 6   blueteam_name            9383 non-null   object
 7   blueteam_code_x          9383 non-null   object
 8   blueteam_win             9383 non-null   int64 
 9   redteam_name             9383 non-null   object
 10  redteam_code_x           9383 non-null   object
 11  redteam_win              9383 non-null   int64 
 12  gameNumberInAMatch       9383 non-null   int64 
 13  gameId                   9383 non-null   object
 14  patch_x                  9383 non-null  

In [79]:
result_df.to_excel("../data/game_ids_merged.xlsx", index=False)

In [80]:
result_with_picks_df = result_df[pd.isna(result_df["pick_0"]) == False].copy()
result_with_picks_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8119 entries, 0 to 9382
Data columns (total 68 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   startTime(match)         8119 non-null   object
 1   tournamentId             8119 non-null   object
 2   blockName                8119 non-null   object
 3   leagueName               8119 non-null   object
 4   matchId                  8119 non-null   object
 5   bestof                   8119 non-null   int64 
 6   blueteam_name            8119 non-null   object
 7   blueteam_code_x          8119 non-null   object
 8   blueteam_win             8119 non-null   int64 
 9   redteam_name             8119 non-null   object
 10  redteam_code_x           8119 non-null   object
 11  redteam_win              8119 non-null   int64 
 12  gameNumberInAMatch       8119 non-null   int64 
 13  gameId                   8119 non-null   object
 14  patch_x                  8119 non-null   obje

In [81]:
result_with_picks_df.to_excel("../data/game_ids_merged_with_picks.xlsx", index=False)

In [82]:
def concat_columns(row):
    return f"{row["blueteam_name"]} vs {row["redteam_name"]}"

In [83]:
def danpan_winnerside(row):
    if row["bestof"] == 1:
        return "Blue" if row["blueteam_win"] == 1 else "Red"
    else:
        return np.nan

In [84]:
handmade_df = result_with_picks_df[pd.isna(result_with_picks_df["winner_side"]) == True].copy()
handmade_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 1638 entries, 181 to 9333
Data columns (total 68 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   startTime(match)         1638 non-null   object
 1   tournamentId             1638 non-null   object
 2   blockName                1638 non-null   object
 3   leagueName               1638 non-null   object
 4   matchId                  1638 non-null   object
 5   bestof                   1638 non-null   int64 
 6   blueteam_name            1638 non-null   object
 7   blueteam_code_x          1638 non-null   object
 8   blueteam_win             1638 non-null   int64 
 9   redteam_name             1638 non-null   object
 10  redteam_code_x           1638 non-null   object
 11  redteam_win              1638 non-null   int64 
 12  gameNumberInAMatch       1638 non-null   int64 
 13  gameId                   1638 non-null   object
 14  patch_x                  1638 non-null   ob

In [85]:
handmade_df["winner_side"] = handmade_df.apply(danpan_winnerside, axis=1)
handmade_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1638 entries, 181 to 9333
Data columns (total 68 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   startTime(match)         1638 non-null   object
 1   tournamentId             1638 non-null   object
 2   blockName                1638 non-null   object
 3   leagueName               1638 non-null   object
 4   matchId                  1638 non-null   object
 5   bestof                   1638 non-null   int64 
 6   blueteam_name            1638 non-null   object
 7   blueteam_code_x          1638 non-null   object
 8   blueteam_win             1638 non-null   int64 
 9   redteam_name             1638 non-null   object
 10  redteam_code_x           1638 non-null   object
 11  redteam_win              1638 non-null   int64 
 12  gameNumberInAMatch       1638 non-null   int64 
 13  gameId                   1638 non-null   object
 14  patch_x                  1638 non-null   ob

In [86]:
real_handmade_df = handmade_df[pd.isna(handmade_df["winner_side"]) == True].copy()
real_handmade_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 883 entries, 288 to 9333
Data columns (total 68 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   startTime(match)         883 non-null    object
 1   tournamentId             883 non-null    object
 2   blockName                883 non-null    object
 3   leagueName               883 non-null    object
 4   matchId                  883 non-null    object
 5   bestof                   883 non-null    int64 
 6   blueteam_name            883 non-null    object
 7   blueteam_code_x          883 non-null    object
 8   blueteam_win             883 non-null    int64 
 9   redteam_name             883 non-null    object
 10  redteam_code_x           883 non-null    object
 11  redteam_win              883 non-null    int64 
 12  gameNumberInAMatch       883 non-null    int64 
 13  gameId                   883 non-null    object
 14  patch_x                  883 non-null    obj

In [87]:
handmade_df.to_excel("../data/handmade_game_ids.xlsx",index = False)

In [88]:
columns = ["startTime(match)", "tournamentId", "teamcode_blue_on_detail", "teamcode_red_on_detail", "winner_side", "gameId", "patch_x", "blueteam_name", "redteam_name"]
columns.extend([f"pick_{i}" for i in range(10)])
columns.extend([f"summonerName_{i}_x" for i in range(10)])
handmade_df_simple = handmade_df.loc[:, columns]
handmade_df_simple["match_vs"] = handmade_df_simple.apply(concat_columns, axis = 1)
handmade_df_simple.drop(["blueteam_name", "redteam_name"], axis=1, inplace=True)
columns_order = ["startTime(match)", "tournamentId", "match_vs", "teamcode_blue_on_detail", "teamcode_red_on_detail", "winner_side", "gameId", "patch_x"]
columns_order.extend([f"pick_{i}" for i in range(10)])
columns_order.extend([f"summonerName_{i}_x" for i in range(10)])
handmade_df_simple = handmade_df_simple[columns_order]

In [89]:
handmade_df_simple.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1638 entries, 181 to 9333
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   startTime(match)         1638 non-null   object
 1   tournamentId             1638 non-null   object
 2   match_vs                 1638 non-null   object
 3   teamcode_blue_on_detail  1638 non-null   object
 4   teamcode_red_on_detail   1638 non-null   object
 5   winner_side              755 non-null    object
 6   gameId                   1638 non-null   object
 7   patch_x                  1638 non-null   object
 8   pick_0                   1638 non-null   object
 9   pick_1                   1638 non-null   object
 10  pick_2                   1638 non-null   object
 11  pick_3                   1638 non-null   object
 12  pick_4                   1638 non-null   object
 13  pick_5                   1638 non-null   object
 14  pick_6                   1638 non-null   ob

In [90]:
handmade_df_simple.to_excel("../data/handmade_game_ids_simple.xlsx", index=None)

In [91]:
real_handmade_df_simple = real_handmade_df.loc[:, columns]
real_handmade_df_simple["match_vs"] = real_handmade_df_simple.apply(concat_columns, axis = 1)
real_handmade_df_simple.drop(["blueteam_name", "redteam_name"], axis=1, inplace=True)
real_handmade_df_simple = real_handmade_df_simple[columns_order]
real_handmade_df_simple.to_excel("../data/real_handmade_game_ids_simple.xlsx", index=None)

In [92]:
real_handmade_df_simple.info()

<class 'pandas.core.frame.DataFrame'>
Index: 883 entries, 288 to 9333
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   startTime(match)         883 non-null    object
 1   tournamentId             883 non-null    object
 2   match_vs                 883 non-null    object
 3   teamcode_blue_on_detail  883 non-null    object
 4   teamcode_red_on_detail   883 non-null    object
 5   winner_side              0 non-null      object
 6   gameId                   883 non-null    object
 7   patch_x                  883 non-null    object
 8   pick_0                   883 non-null    object
 9   pick_1                   883 non-null    object
 10  pick_2                   883 non-null    object
 11  pick_3                   883 non-null    object
 12  pick_4                   883 non-null    object
 13  pick_5                   883 non-null    object
 14  pick_6                   883 non-null    obj